In [68]:
# from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import pickle
from datetime import datetime


In [69]:
    
    #数据输入示例
    # {
    #  "flightNumber": "SQ318",
    #  "departureAirport": "SIN",
    #  "arrivalAirport": "LHR",
    #  "scheduledDeparture": "2023-12-01T12:00:00",
    #  "scheduledArrival": "2023-12-01T18:00:00",
    #  "aircraftType": "Boeing 777",
    #  "gateOrigin": "D32",
    #  "TMAX":20,
    #  "TMIN":10,
    #  "PRCP":25,
    #  "ticketPrice": "1000"
    # }
    
    #初始数据集输入示例，只要省掉不要的值就可以了。之后转为pd.DataFrame格式
    # datainit = {
    #    'scheduledDeparture': scheduled_departure,
    #    'scheduledArrival': scheduled_arrival,
    #    'UniqueCarrier':flight_number,
    #    'TailNum': [3464],
    #    'Origin':departure_airport,
    #    'Dest': arrival_airport,
    # }


In [70]:
    
dataModel1 = pd.DataFrame([{
    'scheduledDeparture':'2023-01-12T07:00:00',
    'scheduledArrival':'2023-01-12T09:55:00' ,
    'UniqueCarrier':'DH',
    'TailNum': 'N421FJ',
    'Origin':'BOS',
    'Dest': 'PHL',
    'TMAX':53,
    'TMIN':-25,
    'PRCP':25,
}])

dataModel2= pd.DataFrame([{
    'scheduledDeparture':'2023-01-12T07:00:00',
    'scheduledArrival':'2023-01-12T09:55:00' ,
    'UniqueCarrier':'DH',
    'TailNum': 'N421FJ',
    'Origin':'BOS',
    'Dest': 'PHL',
    "Price": "1000"
}])




dataModel1['scheduledArrival']=pd.to_datetime(dataModel1['scheduledArrival'])
dataModel1['scheduledDeparture']=pd.to_datetime(dataModel1['scheduledDeparture'])
dataModel2['scheduledArrival']=pd.to_datetime(dataModel2['scheduledArrival'])
dataModel2['scheduledDeparture']=pd.to_datetime(dataModel2['scheduledDeparture'])
    

#Model1    
dataModel1['Month']=dataModel1['scheduledDeparture'].dt.month.astype(int)
dataModel1['DayofMonth']=dataModel1['scheduledDeparture'].dt.day.astype(int)
dataModel1['DepTime']=dataModel1['scheduledDeparture'].dt.hour.astype(int)
dataModel1['ArrTime']=dataModel1['scheduledArrival'].dt.hour.astype(int)

#Model2
dataModel2['Month']=dataModel2['scheduledDeparture'].dt.month.astype(int)
dataModel2['DayofMonth']=dataModel2['scheduledDeparture'].dt.day.astype(int)
dataModel2['DepTime']=dataModel2['scheduledDeparture'].dt.hour.astype(int)
dataModel2['ArrTime']=dataModel2['scheduledArrival'].dt.hour.astype(int)


dataModel1=dataModel1.drop(['scheduledDeparture','scheduledArrival'],axis=1)
dataModel2=dataModel2.drop(['scheduledDeparture','scheduledArrival'],axis=1)
    
    
    # drop_columns=['aircraftType','gateOrigin','weather','ticketPrice']
    
    # datainit=datainit.drop(drop_columns,axis=1,inplace=True)
    
    # Define a function to decode the encoded values back to original

print(dataModel1)
print(dataModel2)

    
with open('encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)
    
with open('encodersTicket.pkl', 'rb') as f:
    encoders1 = pickle.load(f)


  UniqueCarrier TailNum Origin Dest  TMAX  TMIN  PRCP  Month  DayofMonth  \
0            DH  N421FJ    BOS  PHL    53   -25    25      1          12   

   DepTime  ArrTime  
0        7        9  
  UniqueCarrier TailNum Origin Dest Price  Month  DayofMonth  DepTime  ArrTime
0            DH  N421FJ    BOS  PHL  1000      1          12        7        9


In [71]:
        
     #编码和反编码的编译函数   
def encode_category(col, value):
    return encoders[col]['mapping'].get(value, None)

def decode_category(col, encoded_value):
    return encoders[col]['inverse_mapping'].get(encoded_value, None)

def encode_category1(col, value):
    return encoders1[col]['mapping'].get(value, None)

def decode_category1(col, encoded_value):
    return encoders1[col]['inverse_mapping'].get(encoded_value, None)      
        
    
encoder_col=['UniqueCarrier','TailNum','Origin','Dest']
encoder_col1=['UniqueCarrier','TailNum','Origin','Dest']
    
for col in encoder_col:
    dataModel1[col]=[encode_category(col, row) for row in dataModel1[col]]
    
for col in encoder_col1:
    dataModel2[col]=[encode_category1(col, row) for row in dataModel2[col]]
    
new_columns_order=['Month','DayofMonth','DepTime','ArrTime','UniqueCarrier','TailNum','Origin','Dest','TMAX','TMIN','PRCP']
dataModel1=dataModel1[new_columns_order]
new_columns_order1=['Month','DayofMonth','DepTime','ArrTime','UniqueCarrier','TailNum','Origin','Dest','Price']
dataModel2=dataModel2[new_columns_order1]




print(dataModel1)
print(dataModel2)


   Month  DayofMonth  DepTime  ArrTime  UniqueCarrier  TailNum  Origin  Dest  \
0      1          12        7        9              4     1910      32   209   

   TMAX  TMIN  PRCP  
0    53   -25    25  
   Month  DayofMonth  DepTime  ArrTime  UniqueCarrier  TailNum  Origin  Dest  \
0      1          12        7        9              4     1831      26   166   

  Price  
0  1000  


In [72]:
    
with open('flight_modelRaF.pkl', 'rb') as f:
    flight_model = pickle.load(f)
    
with open('ticketModel.pkl', 'rb') as f:
    flight_model1 = pickle.load(f)
    
# 预测

        
y_pred1 = flight_model.predict(dataModel1)


    # 预测逻辑示例
result1 = {
    "ArrDelayPredict": [decode_category('ArrDelay', row) for row in y_pred1[:, 0]],
    "DepDelayPrediction": [decode_category('DepDelay', row) for row in y_pred1[:, 1]],
    "ReasonPrediction":[decode_category('ReasonForDelay', row) for row in y_pred1[:, 2]],
}
    
print(result1)


{'ArrDelayPredict': ['SlightArrDelay'], 'DepDelayPrediction': ['In Time'], 'ReasonPrediction': ['In Time or bit late']}


In [73]:
# print(dataModel2)


y_pred2 = flight_model1.predict(dataModel2)


result2 = pd.DataFrame({
    'FuturePrice': y_pred2
})

print(result2)


   FuturePrice
0  1008.308428


In [74]:
# Example Request:
# {
#   "flightNumber": "SQ318",
#   "departureAirport": "SIN",
#   "arrivalAirport": "LHR",
#   "scheduledDeparture": "2023-12-01T12:00:00",
#   "scheduledArrival": "2023-12-01T18:00:00",
#   "aircraftType": "Boeing 777",
#   "gateOrigin": "D32",
#   "weather": "Clear",
#   "ticketPrice": "1000"
# }
# Responses:

# 	•	Success (200 OK):
# 	•	Body:
#     {
#   "delayPrediction": "Predicted delay: 15 minutes",
#   "ticketPricePrediction": "Predicted ticket price: 120 SGD"
# }

In [75]:
# <!-- <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 120000 entries, 0 to 119999
# Data columns (total 11 columns):
#  #   Column          Non-Null Count   Dtype  
# ---  ------          --------------   -----  
#  0   Month           120000 non-null  int64  
#  1   DayofMonth      120000 non-null  int64  
#  2   DepTime         120000 non-null  float64
#  3   ArrTime         120000 non-null  float64
#  4   UniqueCarrier   120000 non-null  int32  
#  5   TailNum         120000 non-null  int32  
#  8   Origin          120000 non-null  int32  
#  9   Dest            120000 non-null  int32  
# dtypes: float64(2), int32(7), int64(2)
# memory usage: 6.9 MB
# None -->
# 模型输入列示例
